In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
wight2=0
wight1=0

def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))

In [19]:
def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

In [20]:
def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','target']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(len(x))
            if(len(x)>total_records*0):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
    # if total_rule/total_records<0.4:
    #     return (999,);

    avg_confidence = np.sum(rule_confidences) / count +wight1*(entropy_D-entropy_DB)+wight2*(total_rule/total_records)
    return(avg_confidence,)

In [21]:
def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    #random.seed(42) #0.02
    # create individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    top_5_hof = sorted(hof.items, key=lambda x: x.fitness.values[0])[:1]
    for i, ind in enumerate(top_5_hof):
        print(f"Top {i+1} solution: {ind}\nFitness: {ind.fitness.values[0]}\n")
    return top_5_hof
    # return hof

In [22]:
def bestIndividual_KNN(hof,df):
    k = 6
    knn = KNeighborsClassifier(n_neighbors=k)
    for ind in hof:
        df = pd.read_csv('data/bumps/seismic-bumps.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)
        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
        return mean_accuracy,selected_features

In [23]:
def bestIndividual_SVM(hof, df):
    def SVM(trainX, trainY, testX, testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.8, max_iter=500)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY) / testY.shape[0]
        return precision

    maxaccuracy = 0.0
    for ind in hof:
        df = pd.read_csv('data/bumps/seismic-bumps.csv')
        feature_columns = [col for col in df.columns if col not in ['id', 'target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y = df["target"]
        X = df[selected_features]

        # Use stratified k-fold cross-validation
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scores = []
        for train_index, test_index in skf.split(X, Y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index]
            trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
            score = SVM(trainX, trainY, testX, testY)
            scores.append(score)
        avg_score = np.mean(scores)

        if avg_score > maxaccuracy:
            maxaccuracy = avg_score
            best_feature = selected_features

    return maxaccuracy, best_feature

In [24]:
def bestIndividual_DT(hof, df):
    for ind in hof:
        df = pd.read_csv('data/bumps/seismic-bumps.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        X = df[selected_features]
        Y = df['target']
        dt = DecisionTreeClassifier(max_depth=5)
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            dt.fit(X_train, y_train)

            y_pred = dt.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)

        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    return mean_accuracy

In [25]:
wighta=[0,0.1,0.2,0.3,0.4,0.5]
wightb=[0,1,2,3,4,5]
SVM=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
KNN=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
DT=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]
F=[[0 for _ in range(len(wighta))] for _ in range(len(wighta))]


for i_index, i in enumerate(wighta):
    for j_index, j in enumerate(wightb):
        try:
            # 你的代码
            wight1=i
            wight2=j
            n_pop=60
            n_gen=60
            # read dataframe from csv
            df = pd.read_csv('data/bumps/seismic-bumps.csv')
            # encode labels column to numbers
            y = df['target']
            # feature_columns=['f4','f5']
            # feature_columns1=['f6','f7','f17','f18']
            # for col in feature_columns:
            #     df[col] = pd.qcut(df[col], q=20, labels=False, duplicates='drop')
            # feature_columns = [ col for col in df.columns if col not in ['id','target']]
            # for col in feature_columns1:
            #     df[col] = pd.qcut(df[col], q=2, labels=False, duplicates='drop')
            feature_columns = [ col for col in df.columns if col not in ['id','target']]
            X = df[feature_columns]
            individual = [1 for i in range(len(X.columns))]
            getFitness(df,'target',individual)
            print("Accuracy with all features: \t" +
                  str(getFitness(df,'target',individual)) + "\n")
    # apply genetic algorithm
            hof = geneticAlgorithm(df,'target',n_pop, n_gen)

            print('------------wight1--------------')
            print(wight1)
            print('------------wight2--------------')
            print(wight2)

            accuracy_SVM,individual = bestIndividual_SVM(hof,df)
            SVM[i_index][j_index] = accuracy_SVM
            print("-----------SVM---------------")
            print(accuracy_SVM, "保存至 SVM[{}][{}]".format(i_index, j_index))

            accuracy_KNN,individual = bestIndividual_KNN(hof,df)
            KNN[i_index][j_index] = accuracy_KNN
            print("-----------KNN---------------")
            print(accuracy_KNN, "保存至 KNN[{}][{}]".format(i_index, j_index))

            accuracy_DT = bestIndividual_DT(hof,df)
            DT[i_index][j_index] = accuracy_DT
            print("-----------DT---------------")
            print(accuracy_DT, "保存至 DT[{}][{}]".format(i_index, j_index))

            print('------------特征-----------------')
            print(individual)
            F[i_index][j_index] = individual
            print("特征保存至 F[{}][{}]".format(i_index, j_index))

        except Exception as e:
            print("执行过程出现错误：", e)
            continue

Accuracy with all features: 	(0.0,)



C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg       	min
0  	60    	0.00839726	0  
1  	40    	0.0014018 	0  
2  	32    	0         	0  
3  	36    	0.000359281	0  
4  	37    	0          	0  
5  	29    	0          	0  
6  	33    	0.000142622	0  
7  	42    	0.000750533	0  
8  	41    	0.00177313 	0  
9  	32    	0.000467502	0  
10 	39    	0.000424559	0  
11 	32    	9.06618e-05	0  
12 	36    	0          	0  
13 	28    	0          	0  
14 	34    	0.00036336 	0  
15 	33    	0          	0  
16 	33    	0.00018107 	0  
17 	46    	0.000198413	0  
18 	36    	0.000429194	0  
19 	35    	0          	0  
20 	42    	5.54918e-05	0  
21 	34    	6.46747e-06	0  
22 	43    	0          	0  
23 	46    	0          	0  
24 	29    	6.76133e-06	0  
25 	44    	6.76133e-06	0  
26 	33    	9.11577e-05	0  
27 	27    	0          	0  
28 	46    	0.00089696 	0  
29 	25    	0          	0  
30 	36    	0.000482847	0  
31 	39    	0.000354778	0  
32 	38    	1.3344e-05 	0  
33 	31    	6.5152e-05 	0  
34 	37    	0          	0  
35 	32    	0          	0  
36 	3

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

-----------SVM---------------
0.9171814671814673 保存至 SVM[0][0]
-----------KNN---------------
0.9326718745323396 保存至 KNN[0][0]
-----------DT---------------
0.9291760198736944 保存至 DT[0][0]
------------特征-----------------
['f1', 'f3', 'f5', 'f7', 'f10', 'f11', 'f12', 'f16']
特征保存至 F[0][0]
Accuracy with all features: 	(1.0,)



...

-----------SVM---------------
0.9272455179431924 保存至 SVM[5][4]
-----------KNN---------------
0.9326688815060908 保存至 KNN[5][4]
-----------DT---------------
0.9291805094130675 保存至 DT[5][4]
------------特征-----------------
['f2', 'f8', 'f9', 'f12', 'f14', 'f16', 'f17', 'f18']
特征保存至 F[5][4]
Accuracy with all features: 	(5.175005296463443,)



C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg    	min    
0  	60    	5.17654	5.15692
1  	42    	5.1751 	5.15692
2  	38    	5.17197	5.15692
3  	34    	5.17014	5.15692
4  	39    	5.16276	5.15692
5  	31    	5.15869	5.15692
6  	36    	5.15824	5.15692
7  	35    	5.15898	5.15692
8  	38    	5.15763	5.15692
9  	40    	5.15829	5.15692
10 	44    	5.15832	5.15692
11 	42    	5.15979	5.15692
12 	35    	5.159  	5.15692
13 	36    	5.16029	5.15692
14 	32    	5.15854	5.15692
15 	30    	5.15795	5.15692
16 	31    	5.16131	5.15692
17 	37    	5.15848	5.15692
18 	37    	5.15773	5.15692
19 	40    	5.15768	5.15692
20 	42    	5.15811	5.15692
21 	26    	5.15973	5.15692
22 	39    	5.15806	5.15692
23 	39    	5.15866	5.15692
24 	41    	5.16019	5.15692
25 	34    	5.15753	5.15692
26 	38    	5.16257	5.15692
27 	35    	5.16039	5.15692
28 	40    	5.16104	5.15692
29 	31    	5.15899	5.15692
30 	38    	5.15806	5.15692
31 	33    	5.15726	5.15692
32 	27    	5.15768	5.15692
33 	39    	5.15863	5.15692
34 	30    	5.15825	5.15692
35 	38    	5.15723	5.15692
3

C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

-----------SVM---------------
0.9276316183292928 保存至 SVM[5][5]
-----------KNN---------------
0.9326688815060908 保存至 KNN[5][5]
-----------DT---------------
0.9295681063122924 保存至 DT[5][5]
------------特征-----------------
['f2', 'f8', 'f9', 'f13', 'f16', 'f17', 'f18']
特征保存至 F[5][5]
